### Парсер блога baguzin.ru

Блог baguzin.ru посвящен интересным книгам (и не только), которые автор заботливо отбирает для читателя интерсующегося своим интелектуальным развитием. Задача была не спарсить сами статьи о книгах, а только забрать превьюхи с блог-ленты, чтобы не листать ее с самого начала. 

In [1]:
#начинаем с импорта библиотек
import requests
from  bs4 import BeautifulSoup as bs
import csv
from urllib.request import urljoin
from tqdm.notebook import tqdm
import pandas as pd
import re

In [2]:
# Задаем глобальные параметры
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
          }
domen = 'https://baguzin.ru'
url='https://baguzin.ru/wp/category/biblio/'
filename = 'baguzin_books.csv'

In [3]:
def save_product_info(products):
    with open(filename, 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.DictWriter(outfile, fieldnames=get_field_name(products), lineterminator='\n')    
        spamwriter.writeheader() #записывет заголовки столбцов равные ключам словаря
        spamwriter.writerows(products)
        
# Вспом.функция для сбора названия полей с словарях с продуктами для формирования заголовка в CSV (Без сортировки)
def get_field_name(products):
    field_names = set()
    for i in products:
        for k in i:
            field_names.add(k)
    return field_names

In [7]:
# Основной блок
def get_page(pages):
    content=[]
    for url in tqdm(pages, desc='Getting info from blog posts', leave=False):
        session = requests.Session()
        r = session.get(url, headers=headers)
        soup = bs(r.text, 'lxml')
        if soup != None:
            c = soup.find_all('article')
            for i in c:
                article={}
                a = i.find('h2', class_='blog-entry-title entry-title').text
                article['title']=a

                b = i.find('time', )['content']
                article['posted']=b

                d = i.find('a', rel='category tag').text
                article['category']=d

                e = i.find('a')['href']
                article['url']=e

                g = i.find('p').text
                article['text']=g

                try:
                    f = i.find('img')['src']
                    article['img']=f
                except:
                    pass
                content.append(article)
    return content

In [16]:
# Запуск всего парсера
pages = []
pages.append(url)
for i in range(2,58):
    link = f'https://baguzin.ru/wp/category/biblio/page/{i}/'
    pages.append(link)
posts = get_page(pages)
save_product_info(posts)

In [17]:
import pandas as pd
df=pd.read_csv(filename, encoding='utf-8')
#df.sort_index(inplace=True, axis=1)
display(df.shape)
df.head(15)

(562, 6)

,posted,img,category,title,text,url
0,2021-01-06,https://baguzin.ru/wp/wp-content/uploads/2021/...,06. Об экономике,Дэн Ариели. Честно о нечестности: Почему мы лж...,Большинство из нас считает себя честными людьм...,https://baguzin.ru/wp/den-arieli-chestno-o-nec...
1,2021-01-03,https://baguzin.ru/wp/wp-content/uploads/2021/...,07. О маркетинге,Александр Остервальдер. Тестирование бизнес-идей,Это четвертая книга Остервальдера в моей колле...,https://baguzin.ru/wp/aleksandr-ostervalder-te...
2,2020-12-30,https://baguzin.ru/wp/wp-content/uploads/2020/...,02. О вероятностях,"Яков Хургин. Да, нет или может быть… Рассказы ...",Книга известного советского популяризатора нау...,https://baguzin.ru/wp/yakov-hurgin-da-net-ili-...
3,2020-12-26,https://baguzin.ru/wp/wp-content/uploads/2020/...,02. О вероятностях,Юрий Адлер. Введение в планирование эксперимента,Когда я учился в школе (1970-е) мои родители р...,https://baguzin.ru/wp/yurij-adler-vvedenie-v-p...
4,2020-09-16,https://baguzin.ru/wp/wp-content/uploads/2020/...,11. О разном,Fantasy Premier League. Полезные ресурсы,"Это окончание перевода книги Toby Margetts, Gi...",https://baguzin.ru/wp/fantasy-premier-league-p...
5,2020-09-15,NaN,11. О разном,Fantasy Premier League. Глава 9. Управление ва...,"Это продолжение перевода книги Toby Margetts, ...",https://baguzin.ru/wp/fantasy-premier-league-g...
6,2020-09-14,https://baguzin.ru/wp/wp-content/uploads/2020/...,11. О разном,Fantasy Premier League. Глава 8. Психология уп...,"Это продолжение перевода книги Toby Margetts, ...",https://baguzin.ru/wp/fantasy-premier-league-g...
7,2020-09-13,https://baguzin.ru/wp/wp-content/uploads/2020/...,11. О разном,Fantasy Premier League. Глава 7. Использование...,"Это продолжение перевода книги Toby Margetts, ...",https://baguzin.ru/wp/fantasy-premier-league-g...
8,2020-09-11,NaN,11. О разном,Fantasy Premier League. Глава 6. Секрет домини...,"Это продолжение перевода книги Toby Margetts, ...",https://baguzin.ru/wp/fantasy-premier-league-g...
9,2020-09-10,https://baguzin.ru/wp/wp-content/uploads/2020/...,11. О разном,Fantasy Premier League. Глава 5. Самый сильный...,"Это продолжение перевода книги Toby Margetts, ...",https://baguzin.ru/wp/fantasy-premier-league-g...
